In [1]:
# Copyright 2023 DeepMind Technologies Limited
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [2]:
#!pip install --upgrade https://github.com/google-deepmind/nuclease-design.git


  ERROR: HTTP error 404 while getting https://github.com/google-deepmind/nuclease-design.git
ERROR: Could not install requirement https://github.com/google-deepmind/nuclease-design.git because of HTTP error 404 Client Error: Not Found for url: https://github.com/google-deepmind/nuclease-design for URL https://github.com/google-deepmind/nuclease-design.git


In [3]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from nuclease_design import utils
from nuclease_design import plotting
from nuclease_design import constants

ModuleNotFoundError: No module named 'pandas'

# Constants

In [ ]:
RANDOM_SEED = 0
DEFAULT_NUM_BOOTSTRAPS = 1000
RANDOM_STATE = np.random.RandomState(RANDOM_SEED)

In [ ]:
DATA_DIR = constants.DATA_DIR

# Helper fns

In [ ]:
def parse_activity_df_from_xml(activity_path):
  with utils.open_file(activity_path, 'rb', DATA_DIR) as f:
    tree = ET.parse(f)
    root = tree.getroot()

  well_dfs = []
  for sections in root.iter('PlateSection'):
      if (sections.attrib['Name'] != 'Plate1'):
        raise ValueError(f'Unexpected Plate {sections.attrib["Name"]}')
      for well in sections.iter('Well'):
          fluorescence = pd.Series(well.find('RawData').text.split(), dtype=float)
          time = pd.Series(well.find('TimeData').text.split(), dtype=float) / 60  # seconds to minutes
          well = well.get('Name')
          well_dfs.append(pd.DataFrame({'well': well, 'fluorescence': fluorescence, 'time': time}))
  return pd.concat(well_dfs, ignore_index=True)


In [ ]:
# The number of points used here is to guarantee that we are measuring
# initial velocity in the linear regime of the reaction
# See https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4692135/ "Properties and mechanisms of enzyme action"
# for details
def add_initial_velocity(df, num_points=10):
  df = df.sort_values(by=['time'], ascending=True)
  x = df['time'][:num_points]
  y = df['fluorescence'][:num_points]

  result = stats.linregress(x, y, alternative='two-sided')

  slope_stderr = result.stderr
  slope = result.slope
  return pd.Series({'initial_velocity': slope, 'initial_velocity_stderr': slope_stderr, 'intercept': result.intercept})

# The wildtype is a particularly slow reaction, so we have many more points in
# the linear regime.
def add_initial_velocity_wrapper(df):
  if df.genotype.unique().item() == 'wild type':
    return add_initial_velocity(df, num_points=100)
  else:
    return add_initial_velocity(df)

def parametric_resample_row(row, random_state, n_bootstraps):
  mean = row['initial_velocity']
  stdev = row['initial_velocity_stderr']
  return random_state.normal(loc=mean, scale=stdev, size=n_bootstraps)

def get_bootstrapped_initial_velocity_df(df, random_state, n_bootstraps=DEFAULT_NUM_BOOTSTRAPS):
  return df.apply(parametric_resample_row, axis=1, random_state=random_state, n_bootstraps=n_bootstraps)


# Read in plate data

In [ ]:
with utils.open_file(constants.PLATE_GENOTYPES_PATH, 'r', DATA_DIR) as f:
  genotypes_df = pd.read_csv(f)

In [ ]:
platemap = {
    'O403':  ['A1', 'A2', 'A3', 'A4'],
    'O1757': ['C1', 'C2', 'C3', 'C4'],
    'O2360': ['E1', 'E2', 'E3', 'E4'],
    'O3188': ['G1', 'G2', 'G3', 'G4'],
    'O3459': ['I1', 'I2', 'I3', 'I4'],
    'O3512': ['K1', 'K2', 'K3', 'K4'],
    'O3513': ['M1', 'M2', 'M3', 'M4'],
    'O4060': ['O1', 'O2', 'O3', 'O4'],
}

well_to_strain = {}
for strain, wells in platemap.items():
  for well in wells:
    well_to_strain[well] = strain

well_to_concentration = {}
for row in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'M', 'O']:
  for concentration, column in zip([100, 50, 25, 12.5], ['1', '2', '3', '4']):
    well_to_concentration[''.join([row, column])] = concentration

In [ ]:
df = parse_activity_df_from_xml(constants.TIME_SERIES_ACTIVITY_XML_PATH)
df['strain'] = df['well'].apply(well_to_strain.get)
df.dropna(subset=['strain'], inplace=True)
df = df[df['strain'] != 'O403']  # Removing WT assessed in a different background strain
df = df.merge(genotypes_df, how='left', on='strain')
df['concentration'] = df['well'].apply(well_to_concentration.get)

# Compute enzyme reaction rate

## Example of fitting reaction trajectory in linear regime

In [ ]:
slope = add_initial_velocity(df[df.well == 'K1'])['initial_velocity']
stderr = add_initial_velocity(df[df.well == 'K1'])['initial_velocity_stderr']
intercept = add_initial_velocity(df[df.well == 'K1'])['intercept']

In [ ]:
plotdf = df[(df['well'] == 'K1')]
sns.scatterplot(data=plotdf, x='time', y='fluorescence', s=4)
plt.ylim(bottom=0)
plt.plot(plotdf.time, intercept + slope * plotdf.time, 'black', alpha=0.5, label='initial velocity fit')
plt.title(f'{plotdf.genotype.iloc[0]} conc {plotdf.concentration.iloc[0]} well {plotdf["well"].unique()}')
plt.show()

## Add bootstrapped estimates of the rate

In [ ]:
initial_velocity_df = df.groupby(['strain', 'genotype', 'concentration'], group_keys=True).apply(add_initial_velocity_wrapper).reset_index()
initial_velocity_df['initial_velocity_resampled'] = get_bootstrapped_initial_velocity_df(initial_velocity_df, RANDOM_STATE)
bootstrapped_initial_velocity_df = initial_velocity_df.explode('initial_velocity_resampled')

In [ ]:
bootstrapped_initial_velocity_df.head()

# Purified Protein Plot

In [ ]:
genotype_to_library = {
    'wild type': 'WT',
    'A73R': 'epPCR',
    'A73R,D74S': 'ML2',
    'A63P,A73R,D74H,I84Y': 'ML3',
    'A63S,D64S,A73R': 'DE3',
    'D64A,A73R': 'ML2B',  # Also ML2
    'A63S,A132P': 'DE3B', # Internal DE. (sorted from OB174)
}
genotype_order = ('wild type', 'A73R', 'A73R,D74S', 'A63S,D64S,A73R', 'A63P,A73R,D74H,I84Y')
library_order = tuple(genotype_to_library[genotype] for genotype in genotype_order)

custom_palette = (plotting.WT_COLOR, plotting.EPPCR_COLOR, plotting.ML2_COLOR, plotting.DE3_COLOR, plotting.ML3_COLOR)

In [ ]:
# add fields
bootstrapped_initial_velocity_df['activity'] = bootstrapped_initial_velocity_df['initial_velocity_resampled']
bootstrapped_initial_velocity_df['library'] = bootstrapped_initial_velocity_df.genotype.apply(genotype_to_library.get)

# remove other genotypes
bootstrapped_initial_velocity_df = bootstrapped_initial_velocity_df[bootstrapped_initial_velocity_df.genotype.isin(genotype_order)]

In [ ]:
bootstrapped_initial_velocity_df.genotype.unique()

In [ ]:
def get_activity_relative_to_wt(df):
  wt_activities = df[df.genotype == 'wild type']['activity'].sample(n=len(df.activity), replace=True)
  df['fold_change_activity'] = df['activity'] / wt_activities.values
  return df

In [ ]:
normalized_df = bootstrapped_initial_velocity_df.groupby(by='concentration', group_keys=False).apply(get_activity_relative_to_wt)

In [ ]:
plt.figure(figsize=(8, 6))
ax = plotting.plot_purified_protein_activity(
    normalized_df,
    library_order=library_order,
    custom_palette=custom_palette,
)

points_df = (
    normalized_df.groupby(by=['genotype', 'concentration'])
    .agg({'fold_change_activity': 'mean'})
    .reset_index()
)
points_df['library'] = points_df.genotype.apply(genotype_to_library.get)
sns.stripplot(
    data=points_df,
    x='library',
    y='fold_change_activity',
    order=library_order,
    marker='o',
    facecolors='black',
    edgecolor='black',
    linewidth=1,
    alpha=0.6,
    ax=ax,
)
plt.xlabel('')

In [ ]:
plt.figure(figsize=(6, 4))
ax = sns.lineplot(
    data=normalized_df,
    x='concentration',
    y='activity',
    hue='library',
    errorbar='sd',
    err_style='band',
    hue_order=library_order,
    marker='o',
    palette=custom_palette,
)
sns.move_legend(ax, 'upper left', bbox_to_anchor=(1, 1))
ax.get_legend().set_title('Best Strain')

In [ ]:
plt.figure(figsize=(6, 4))
ax = sns.lineplot(
    data=normalized_df,
    x='concentration',
    y='fold_change_activity',
    hue_order=library_order,
    errorbar='sd',
    err_style='band',
    hue='library',
    marker='o',
    palette=custom_palette,
)
plt.legend(bbox_to_anchor=[1.0, 1.05])
ax.get_legend().set_title('Best Strain')

# Permutation test

Test whether the fold_change_activity is greater for ML3 than DE3

In [ ]:
initial_velocity_df = df.groupby(['strain', 'genotype', 'concentration'], group_keys=True).apply(add_initial_velocity_wrapper).reset_index()
initial_velocity_df['activity'] = initial_velocity_df['initial_velocity']
initial_velocity_df['library'] = initial_velocity_df.genotype.apply(genotype_to_library.get)
normalized_no_bootstrap_df = initial_velocity_df.groupby(by='concentration', group_keys=False).apply(get_activity_relative_to_wt)

In [ ]:
normalized_no_bootstrap_df[normalized_no_bootstrap_df['library'].isin(['DE3', 'ML3', 'WT'])][['library', 'concentration', 'fold_change_activity']]

In [ ]:
de3_fold_changes = normalized_no_bootstrap_df[normalized_no_bootstrap_df['library'] == 'DE3']['fold_change_activity'].values

In [ ]:
ml3_fold_changes = normalized_no_bootstrap_df[normalized_no_bootstrap_df['library'] == 'ML3']['fold_change_activity'].values

In [ ]:
def statistic(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

statistic(de3_fold_changes, ml3_fold_changes, axis=0)

In [ ]:
# Taken from https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.permutation_test.html
# because our statistic is vectorized, we pass `vectorized=True`
# `n_resamples=np.inf` indicates that an exact test is to be performed
res = stats.permutation_test(
    (de3_fold_changes, ml3_fold_changes),
    statistic,
    vectorized=True,
    n_resamples=np.inf,
    alternative='less',
)

In [ ]:
print(res.statistic)

In [ ]:
print(res.pvalue)

In [ ]:
stats.ttest_ind(de3_fold_changes, ml3_fold_changes)

## Account for uncertainty in slope estimation

We can do the same permutation test on our parametric bootstrap resamples of

  $$ \dfrac{\text{initial_velocity_A}} {\text{initial_velocity_WT}} $$

  where both the distribution of the numerator and denominator are modeled as Gaussian

In [ ]:
sns.histplot(data=normalized_df[normalized_df['library'].isin(['DE3', 'ML3'])], hue='library', x='fold_change_activity')

In [ ]:
de3_fold_changes_resampled = normalized_df[normalized_df['library'] == 'DE3']['fold_change_activity'].values

In [ ]:
ml3_fold_changes_resampled = normalized_df[normalized_df['library'] == 'ML3']['fold_change_activity'].values

In [ ]:
for n_resamples in [10, 100, 1000]:
  res = stats.permutation_test(
      (de3_fold_changes_resampled, ml3_fold_changes_resampled),
      statistic,
      vectorized=True,
      n_resamples=n_resamples,
      alternative='less',
      permutation_type='independent'
  )
  n, _, _ = plt.hist(res.null_distribution)
  plt.title(f'resamples: {n_resamples} statistic: {res.statistic:.2f} pvalue: {res.pvalue:0.2e}')
  plt.vlines(x=res.statistic, ymin=0, ymax=np.max(n))
  plt.show()

This dependence on the number of permutation test resamples is due to the test statistic being so significant that our numerical approximation of the p-value breaks down. In any case, this is clearly a significant result, and there's no reason to push any further on it

# Appendix

In [ ]:
def make_initial_velocity_plot(df, genotype, concentration):
    genotype_df = df[
        (df['genotype'] == genotype) & (df['concentration'] == concentration)
    ]
    wtdf = df[
        (df['genotype'] == 'wild type') & (df['concentration'] == concentration)
    ]
    genotype_slope = add_initial_velocity_wrapper(genotype_df)['initial_velocity']
    genotype_intercept = add_initial_velocity_wrapper(genotype_df)['intercept']

    wt_slope = add_initial_velocity_wrapper(wtdf)['initial_velocity']
    wt_intercept = add_initial_velocity_wrapper(wtdf)['intercept']

    sns.scatterplot(
        data=pd.concat([genotype_df, wtdf]),
        x='time',
        y='fluorescence',
        s=8,
        alpha=0.5,
        hue='genotype',
        hue_order=['wild type', genotype],
    )
    plt.legend()

    wt_args = (wt_slope, wt_intercept, wtdf)
    genotype_args = (genotype_slope, genotype_intercept, genotype_df)
    for i, args in enumerate([wt_args, genotype_args]):
      slope, intercept, plotdf = args
      plt.plot(
          plotdf.time,
          intercept + slope * plotdf.time,
          alpha=0.2,)

    plt.ylim(bottom=0)
    plt.title(
        f'{genotype} conc {concentration} ng/mL well'
        f' {genotype_df["well"].unique()}'
    )
    plt.xlabel('time (min)')
    plt.xlim(left=0, right=40)
    # 2 minutes is what we used for slope estimation, 20 minutes is what we used for WT
    plt.xticks([0, 2, 10, 20, 30, 40])
    plt.show()


In [ ]:
for genotype in df['genotype'].unique():
  for concentration in df['concentration'].unique():
    if genotype == 'wild type':
      continue
    with sns.axes_style('ticks'):
      make_initial_velocity_plot(df, genotype, concentration)

In [ ]:
# Todo - add wt, DE3, ML3 fluorescence plots

# Appendix

## Provenance of top performers

In [ ]:
all_data_df = utils.load_all_data()

In [ ]:
def get_mutation_string_from_tuple(mutation_tuples):
    if len(mutation_tuples) == 0:
      return ''
    else:
      return ','.join([''.join([str(t) for t in mutation_tuple]) for mutation_tuple in mutation_tuples])

genotype_to_mutation_tuple = {
    'wild type': (),
    'A73R': (('A', 73, 'R')),
    'A73R,D74S': (('A', 73, 'R'), ('D', 74, 'S')),
    'A63P,A73R,D74H,I84Y': (('A', 63, 'P'), ('A', 73, 'R'), ('D', 74, 'H'), ('I', 84, 'Y')),
    'A63S,D64S,A73R': (('A', 63, 'S'), ('D', 64, 'S'), ('A', 73, 'R')),
    'D64A,A73R': (('D', 64, 'A'), ('A', 73, 'R')),
    'A63S,A132P': (('A', 63, 'S'), ('A', 132, 'P'))}


In [ ]:
initial_velocity_df['mutations'] = initial_velocity_df.genotype.apply(genotype_to_mutation_tuple.get)

In [ ]:
initial_velocity_df.merge(all_data_df, on='mutations', how='inner').groupby('mutations').aggregate('first')